# Measure Limbs

The aim of this notebook is to automatically take the same measurements as in the 1D dataset on the limbs created using the SSM. What this is going to lead to is the creation of a ML pipeline which will take in measurements and output the components of the SSM that get closest to that particular set of measurements (Also lets include an output for scale). Inshallah.

I think the approach I'm going to take for the circumference measurements is fairly simple - define a plane based on a vertical normal and a selected vertex. For every vert figure out which side of the plane its on. Find edges which cut the plane (And vertices on the plane). Find the intersection points between the edge and the plane. Join points based on face adjaceny - et voila. A curve on the plane. Then its just a matter of adding up all the line segments in a loop and bobs your uncle you have the circumfrential measurement. Maybe there is a nicer way to do it with a half edge mesh but I really haven't played about with that too much. 

As for the lengths: Just pick to verts and off you go. Nothing complicated about that at all.

In [24]:
import igl
import numpy as np
from matplotlib import pyplot as plt

verts, face2vert = igl.read_triangle_mesh("stls/limb_00000.stl")
edge2vert, face2edge, edge2face = igl.edge_topology(verts, face2vert)

## Slicing out a curve with a plane
So we're going to try get out a 2d curve out of this mesh. So one of the things I currently cant remember how to do is find where this edge intersects a plane but lets work it out because thats fun. 

We start with a point on the edge which can be described as

$$p_t = tv_1 + (1 - t)v_2$$

We then want to find when the vector from this to the plane point is orthogonal so: 

$$(p_t - p_p) \cdot n_p = 0$$

Which when you look at it you realise is really fucking easy because in the case of a vertical normal vector you end up with making the z value zero obviously. But if the plane wasnt horizontal then you'd just have to solve that equation and everyone will be happy. Great. So in the case of horizontal planes I should be solving: 

$$v_{2z} + (v_{1z} - v_{2z})t = 0$$

$$\frac{v_{2z}}{v_{2z} - v_{1z}} = t$$

In [ ]:
plane_normal = np.array([0, 0, 1])
plane_point = np.array([0, 0, 0])

plane_side = np.dot(verts - plane_point, plane_normal) < 0
# If a vert sits exactly on the plane then make it not and itll sort itself out later
plane_side[plane_side == 0] = 1e-8

edge_verts_side = plane_side[edge2vert]
edges_cross_plane = np.where(edge_verts_side[:, 0] != edge_verts_side[:, 1])[0]

edge_cut_verts = verts[edge2vert[edges_cross_plane]]

v1 = edge_cut_verts[:, 0]
v2 = edge_cut_verts[:, 1]
d1 = np.dot(v1 - plane_point, plane_normal)
d2 = np.dot(v2 - plane_point, plane_normal)
t = d1 / (d1 - d2)
intersection_points = v1 + (v2 - v1) * t[:, np.newaxis]

intersection_points.shape

(784, 3)

In [42]:
edge_set = set(edges_cross_plane)

np.isin(face2edge[edge2face[edges_cross_plane]], tuple(edge_set))


var = face2edge[edge2face[edges_cross_plane]]
var[var != edges_cross_plane[:, None, None]].shape

(3920,)

In [ ]:
with open("test_cuts.obj", "w") as f:
    for v1, v2, v3 in intersection_points:
        f.write(f"v {v1} {v2} {v3}\n")